In [1]:
import operator
import re

from pyspark import SparkConf, SparkContext

In [2]:
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [3]:
def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return [word.lower() for word in words]
    except ValueError as e:
        return []

In [4]:
articles = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)

In [5]:
def process_article(words):
    prew = words[0]
    for word in words[1:]:
        yield (prew, word), 1
        prew = word

In [6]:
word_count = articles.flatMap(process_article).reduceByKey(operator.add)

In [7]:
narodnaya = word_count.filter(lambda item: item[0][0] == 'narodnaya').sortByKey()

In [8]:
for row in narodnaya.collect():
    print '{key[0]}_{key[1]}\t{value}'.format(key=row[0], value=row[1])

narodnaya_gazeta	1
narodnaya_volya	9
